# Check p=0

TJ Kim

6.19.21

### Objective:
- Check p=0 case for offloading decisions, the distance x load should be equivalent for everybody.
- Is there a rounding error?


In [1]:
cd /home/tkim/Academics/DistMEC

/home/tkim/Academics/DistMEC


In [2]:
import numpy as np
import math
import copy
import itertools

from classes.Application import *
from classes.User import *
from classes.Server import *
from solver.Sim_Params import *
from classes.Central_Controller import *
from classes.Container import *
from solver.run_sim_temp import *

In [35]:
sim_param = Sim_Params(big_ts=5, small_ts=5, x_length=5, y_length=5, num_users=20, num_servers=10, num_apps=5)
sim_param.dist_p = 0
sim_param.deploy_rounds = 8
sim_param.app_mode = "uniform"
servers, users, containers, apps = setup_sim(sim_param)

In [95]:
sim_param.dist_n = 0.01
sim_param.dist_p = 1
sim_param.app_mode = "dist"
# Loop through big time step
cc = Central_Controller(servers, containers, sim_param, apps, users)
cc.mode = sim_param.cc_mode
cc_deployment_history = {}
cc_util_history = {}

# Clear container memory
for c_key in containers.keys():
    containers[c_key].flush_queue()
    containers[c_key].reset_history()

for bt in range(sim_param.big_ts):

    # Deploy the containers to the servers
    cc.big_ts = bt
    cc.VM_placement(users,apps,sim_param.deploy_rounds)
    cc_deployment_history[bt] = cc.container_deployed
    _,_,_,temp_util_hist = cc.compute_container_utility(cc.container_deployed,users,apps,sim_param.app_mode)
    cc_util_history[bt] = np.sum(np.sum(temp_util_hist,axis=1))
    
    # Reset containers to rid of queues
    for c_key in containers.keys():
        containers[c_key].flush_queue()

    # For each small time step offload and serve at container
    for st in range(sim_param.small_ts):
        # random order between users when offloading for each app
        cc.small_ts = st
        usr_order = np.arange(len(users))
        np.random.shuffle(usr_order)

        temp_containers = {}
        queue_replies = {}

        # Make offloading decision
        for u in usr_order:
            # Generate load
            # apps[u].new_load(ts_big=bt,ts_small=st)
            # Decide to offload given servers --> add offload policy to app class
            offload_u = apps[u].offload(cc.container_deployed, bt, st, cc)
            for (s,a) in offload_u.keys():
                if (s,a) not in temp_containers:
                    temp_containers[(s,a)] = np.empty([0,4])
                temp_containers[(s,a)] = np.append(temp_containers[(s,a)],offload_u[(s,a)],axis=0)

        # Scramble arrived job and add to queue, apps record latency
        for (s,a) in temp_containers.keys():
            sa_offload = temp_containers[(s,a)]
            np.random.shuffle(sa_offload) 
            replies = containers[(s,a)].add_to_queue(sa_offload)
            # print(containers[(s,a)].queue)
            queue_replies[(s,a)] = replies

            for i in range(replies.shape[0]):
                # Add distance between app and server
                a_id, reply_len, load = int(replies[i,0]), replies[i,2], replies[i,1]
                dist = cc.server_dists[int(users[a_id].user_voronoi_true[bt]),s]
                apps[a_id].record_queue_length(reply_len, s, bt, st, load, dist)

            # Service the queue
            containers[(s,a)].serve_ts()

In [96]:
cc_deployment_history

{0: array([[1., 0., 0., 0., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 1: array([[0., 0., 1., 1., 0., 0., 0., 1., 0., 0.],
        [1., 1., 1., 0., 1., 1., 1., 0., 1., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0., 0., 1.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 2: array([[1., 0., 0., 1., 0., 0., 0., 1., 1., 0.],
        [1., 1., 1., 0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 1., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 3: array([[0., 0., 0., 1., 1., 0., 0., 1., 1., 0.],
        [1., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.,

In [97]:
cc_util_history

{0: -21.265721166505713,
 1: -19.15688248986399,
 2: -22.22031641408463,
 3: -21.095710875833664,
 4: -23.15523574064467}

In [93]:
cc_deployment_history

{0: array([[1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 1: array([[1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 1., 1., 1., 1., 1., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 2: array([[1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 3: array([[1., 0., 1., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.,

In [94]:
cc_util_history

{0: -39.24585005981713,
 1: -39.90883283425103,
 2: -40.10082301337313,
 3: -43.47241571803089,
 4: -42.4124462641332}